In [127]:
from sklearn import cross_validation, grid_search, linear_model,svm, metrics,preprocessing

import numpy as np
import pandas as pd

In [3]:
raw_data = pd.read_csv('train.csv', header = 0, sep = ',')

In [96]:
raw_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [100]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [10]:
raw_data.isnull().values.any()

True

In [97]:
target=raw_data[['Survived']]
target=target.values.ravel()
data=raw_data[['Pclass','SibSp','Parch','Fare']]
data['Sex']=raw_data.Sex.apply( lambda x:0 if x == 'male' else 1 )
data['Age']=raw_data['Age'].fillna(raw_data['Age'].mean())

C:\Users\Evtisov_SS\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Evtisov_SS\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [122]:
data[:10]

,Pclass,SibSp,Parch,Fare,Sex,Age
0,3,1,0,7.2500,0,22.000000
1,1,1,0,71.2833,1,38.000000
2,3,0,0,7.9250,1,26.000000
3,1,1,0,53.1000,1,35.000000
4,3,0,0,8.0500,0,35.000000
5,3,0,0,8.4583,0,29.699118
6,1,0,0,51.8625,0,54.000000
7,3,3,1,21.0750,0,2.000000
8,3,0,2,11.1333,1,27.000000
9,2,1,0,30.0708,1,14.000000


## Обучение на отложенной выборке

### LogisticRegression

In [129]:
train_data, test_data, train_labels, test_labels=cross_validation.train_test_split(data, target,test_size = 0.3,random_state = 0)
scaler = preprocessing.StandardScaler().fit(train_data)
train_data=scaler.transform(train_data)
test_data=scaler.transform(test_data)
#logistic
log_regressor = linear_model.LogisticRegression(random_state = 1)
log_regressor.fit(train_data, train_labels)
log_acc=metrics.accuracy_score(test_label, log_regressor.predict(test_data))

#ridge
ridge_classifier = linear_model.RidgeClassifier(random_state = 1)
ridge_classifier.fit(train_data, train_labels)
ridge_acc=metrics.accuracy_score(test_label, ridge_classifier.predict(test_data))

#LinearSVC
lin_clf = svm.LinearSVC(random_state=1)
lin_clf.fit(train_data, train_labels)
lin_clf_acc=metrics.accuracy_score(test_label, ridge_classifier.predict(test_data))

# Выводим точность классификатора
print ("Log acc=%s coef=%s"%(log_acc,log_regressor.coef_))
print ("Ridge acc=%s coef=%s"%(ridge_acc,ridge_classifier.coef_))
print ("Lin_clf acc=%s coef=%s"%(lin_clf_acc,lin_clf.coef_))

Log acc=0.80223880597 coef=[[-0.85339103 -0.47122057 -0.09765102  0.10580759  1.31016329 -0.51416924]]
Ridge acc=0.783582089552 coef=[[-0.26990518 -0.11205587 -0.03426202  0.04023039  0.49319348 -0.15473116]]
Lin_clf acc=0.783582089552 coef=[[-0.29244501 -0.15489252 -0.03507414  0.01796014  0.50986002 -0.17400138]]


### CRoss Validation

In [119]:
# Добавить pipeline со scalar
def cv_run(): 
    cv_strategy = cross_validation.StratifiedShuffleSplit(target, n_iter = 20 , test_size = 0.3, random_state = 2)
    lr_scoring = cross_validation.cross_val_score(log_regressor, data, target, scoring = 'accuracy', cv = cv_strategy)
    ridge_scoring = cross_validation.cross_val_score(ridge_classifier, data, target, scoring = 'accuracy', cv = cv_strategy)
    print ("Cross validation:")
    print (' Ridge mean:{}, max:{}, min:{}, std:{}'.format(ridge_scoring.mean(), ridge_scoring.max(), 
                                                     ridge_scoring.min(), ridge_scoring.std()))
    print (' Log mean:{}, max:{}, min:{}, std:{}'.format(lr_scoring.mean(), lr_scoring.max(), 
                                                   lr_scoring.min(), lr_scoring.std()))

In [120]:
cv_run()

Cross validation:
 Ridge mean:0.8014925373134328, max:0.8395522388059702, min:0.75, std:0.02158381322982026
 Log mean:0.8055970149253732, max:0.8432835820895522, min:0.7611940298507462, std:0.019951349828798287


Ridge mean:0.682276119402985, max:0.7201492537313433, min:0.6455223880597015, std:0.021476303726247585


Log mean:0.6805970149253732, max:0.7164179104477612, min:0.6417910447761194, std:0.020788639219677794
